In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime as dt
import time
import json
from pandas.io.json import json_normalize
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager
option = webdriver.ChromeOptions()
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--disable-dev-shm-usage')
option.add_argument("user-agent=Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; Lumia 640 XL LTE) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Mobile Safari/537.36 Edge/12.10166")
driver = webdriver.Chrome(ChromeDriverManager(version='95.0.4638.69').install(), options = option)
date_now = dt.now().strftime('%Y-%m-%d')
url='https://geoportal.lampungprov.go.id/gis/apps/webappviewer/index.html?id=1264d172d3a34d59a99210733672695e'
link ='https://geoportal.lampungprov.go.id/gis/rest/services/Dinas_Kesehatan/Covid/FeatureServer/0/6/query?where=objectid+%3D+objectid&outfields=*&f=json'
prov_link = 'https://geoportal.lampungprov.go.id/gis/home/webmap/viewer.html?url=https%3A%2F%2Fgeoportal.lampungprov.go.id%2Fgis%2Frest%2Fservices%2FDinas_Kesehatan%2FCovid%2FFeatureServer&source=sd'
driver.get(link)
time.sleep(30)



====== WebDriver manager ======
Driver [/Users/gitasafitri/.wdm/drivers/chromedriver/mac64/95.0.4638.69/chromedriver] found in cache


In [2]:
import urllib3
urllib3.disable_warnings()
content = driver.page_source
soup = BeautifulSoup(content, 'lxml').text
ne = json.loads(str(soup))
a1 = json_normalize(ne['feature'])
df1 = pd.DataFrame(a1)

<ipython-input-2-3eb99c211d80>:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  a1 = json_normalize(ne['feature'])


In [3]:
df1

,geometry.rings,attributes.objectid,attributes.kode_unsur,attributes.kabupaten,attributes.sumber,attributes.kab_kota,attributes.sumber_dat,attributes.ks_baru,attributes.ks_lama,attributes.ks_total,...,attributes.kmk_baru,attributes.kmk_lama,attributes.kmk_total,attributes.st_area_sh,attributes.st_length_,attributes.ks_dalam_p,attributes.pp_skrinin,attributes.status,attributes.resiko,attributes.analisa
0,"[[[104.95119981691903, -4.527825737167234], [1...",4,40404,KABUPATEN LAMPUNG UTARA,Peta RBI Skala 1:25000,Lampung Utara,Dinas Kesehatan,0,147,0,...,0,1,176,0.206371,2.987748,0,69,Terdampak,Resiko Rendah,Kasus tetap


In [4]:
df1 = df1.rename(columns={
    'attributes.kabupaten':'kabkot',
    'attributes.ks_total':'total_suspek',
    'attributes.kp_total':'total_probable',
    'attributes.ke_total':'total_odp',
    'attributes.ke_dalam_p':'odp_dipantau',
    'attributes.kk_total':'total_positif',
    'attributes.kmk_total':'positif_meninggal',
    'attributes.selesai_is':'positif_sembuh',
    })
df1[["total_suspek", "total_probable"]] = df1[["total_suspek", "total_probable"]].apply(pd.to_numeric)
df1["total_pdp"]=df1.apply(lambda x: x['total_suspek'] + x['total_probable'], axis=1)
df=df1[['kabkot','total_odp','odp_dipantau','total_pdp','total_positif','positif_meninggal','positif_sembuh']]

In [5]:
df

,kabkot,total_odp,odp_dipantau,total_pdp,total_positif,positif_meninggal,positif_sembuh
0,KABUPATEN LAMPUNG UTARA,28,196,13,3951,176,3528


In [6]:
driver.get(url)
time.sleep(20)
cont = driver.page_source
soup1 = BeautifulSoup(cont, 'lxml')
all=soup1.findAll('body')

In [7]:
All=[]
data=[]
for i in range(len(all)) :
    All.append(all[i].findAll('b'))
    for j in range(len(All[i])):
        data.append(All[i][j].get_text(strip=True).split())
data

[['DASHBOARD', 'PEMANTAUAN', 'COVID-19', 'PROVINSI', 'LAMPUNG'],
 ['diupdate',
  'oleh',
  ':',
  'DINAS',
  'KESEHATAN',
  'PROVINSI',
  'LAMPUNG',
  'tanggal',
  '17'],
 [],
 ['Desember', '2021']]

In [8]:
tanggal=data[1][8]
bulan = ['januari', 'februari', 'maret', 'april', 'mei', 'juni','juli','agustus', 'september', 'oktober','november','desember']
bln = bulan.index(data[3][0].lower())+1
date_update =data[3][1]+'-'+str(bln)+'-'+tanggal
date_update=dt.strptime(date_update,'%Y-%m-%d').strftime('%Y-%m-%d')
date_update

'2021-12-17'

In [9]:
df = pd.DataFrame(df)
df.loc[:, "scrape_date"]= date_now
df.loc[:, "date_update"]=date_update
df.loc[:,"provinsi"]="LAMPUNG" 
df.loc[:,"odp_sembuh"]=0
df.loc[:,"odp_meninggal"]=0
df.loc[:, "pdp_dipantau"]=0
df.loc[:, "pdp_sembuh"]=0
df.loc[:,"types"]="kabkot"
df.loc[:,"source_link"]=prov_link
df.loc[:,"user_pic"]="Tubagus Ibrohim"
df = df[['scrape_date','date_update','provinsi','kabkot','total_odp','total_pdp','pdp_sembuh','pdp_dipantau','odp_sembuh','odp_dipantau','odp_meninggal','total_positif','positif_sembuh','positif_meninggal','source_link','user_pic']]
df.head()


,scrape_date,date_update,provinsi,kabkot,total_odp,total_pdp,pdp_sembuh,pdp_dipantau,odp_sembuh,odp_dipantau,odp_meninggal,total_positif,positif_sembuh,positif_meninggal,source_link,user_pic
0,2021-12-19,2021-12-17,LAMPUNG,KABUPATEN LAMPUNG UTARA,28,13,0.0,0.0,0.0,196,0.0,3951,3528,176,https://geoportal.lampungprov.go.id/gis/home/w...,Tubagus Ibrohim


In [8]:
import mysql.connector
from mysql.connector import errorcode
from sqlalchemy import create_engine

try:
    mydb = mysql.connector.connect(host='db-blanja2',
                                      port=3306,
                                 user='covid_user',
                                 passwd='5bb6593aa078',
                                 database='covid')
    cursor = mydb.cursor()
    cols2 = "`,`".join([str(i) for i in df.columns.tolist()])
    # Insert DataFrame records one by one. 
    for i,row in df.iterrows():
        sql = "INSERT INTO `covid19_data` (`" +cols2 + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)" 
        cursor.execute(sql, tuple(row)) 
        # the connection is not autocommitted by default, so we must commit to save our # changes 
        mydb.commit()

except mysql.connector.Error as error:
    print("Failed to insert into MySQL table {}".format(error))

finally:
    if (mydb.is_connected()):
        cursor.close()
        mydb.close()
        print("Provinsi Lampung Done " +date_now)

Provinsi Lampung Done 2021-08-30


In [9]:
import pymysql.cursors
import sqlalchemy

engine = sqlalchemy.create_engine('mysql+pymysql://covid_user5bb6593aa078@db-blanja2:3306/covid')
